In [1]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: autogenes 1.0.4 has a non-standard dependency specifier matplotlib>=3.0.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of autogenes or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [10]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from datasets import load_dataset
from tqdm import tqdm
from transformers import BertTokenizerFast, BertForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset, load_metric


In [11]:
dataset = load_dataset("squad_v2")

In [12]:
dataset["train"][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [13]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


In [14]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [15]:
show_random_elements(dataset["train"])

,id,title,context,question,answers
0,5727a9504b864d19001639cf,A_cappella,"A strong and prominent a cappella tradition was begun in the midwest part of the United States in 1911 by F. Melius Christiansen, a music faculty member at St. Olaf College in Northfield, Minnesota. The St. Olaf College Choir was established as an outgrowth of the local St. John's Lutheran Church, where Christiansen was organist and the choir was composed, at least partially, of students from the nearby St. Olaf campus. The success of the ensemble was emulated by other regional conductors, and a rich tradition of a cappella choral music was born in the region at colleges like Concordia College (Moorhead, Minnesota), Augustana College (Rock Island, Illinois), Wartburg College (Waverly, Iowa), Luther College (Decorah, Iowa), Gustavus Adolphus College (St. Peter, Minnesota), Augustana College (Sioux Falls, South Dakota), and Augsburg College (Minneapolis, Minnesota). The choirs typically range from 40 to 80 singers and are recognized for their efforts to perfect blend, intonation, phrasing and pitch in a large choral setting.",Who did the St. Olaf Choir influence?,"{'text': ['other regional conductors'], 'answer_start': [468]}"
1,5731185a05b4da19006bcd90,Bird,"All modern birds lie within the crown group Aves (alternately Neornithes), which has two subdivisions: the Palaeognathae, which includes the flightless ratites (such as the ostriches) and the weak-flying tinamous, and the extremely diverse Neognathae, containing all other birds. These two subdivisions are often given the rank of superorder, although Livezey and Zusi assigned them ""cohort"" rank. Depending on the taxonomic viewpoint, the number of known living bird species varies anywhere from 9,800 to 10,050.",All modern birds lie within which crown group?,"{'text': ['Aves'], 'answer_start': [44]}"
2,5a749a4a42eae6001a389a14,Green,"In surveys made in Europe and the United States, green is the color most commonly associated with nature, life, health, youth, spring, hope and envy. In Europe and the U.S. green is sometimes associated with death (green has several seemingly contrary associations), sickness, or the devil, but in China its associations are very positive, as the symbol of fertility and happiness. In the Middle Ages and Renaissance, when the color of clothing showed the owner's social status, green was worn by merchants, bankers and the gentry, while red was the color of the nobility. The Mona Lisa by Leonardo da Vinci wears green, showing she is not from a noble family; the benches in the British House of Commons are green, while those in the House of Lords are red. Green is also the traditional color of safety and permission; a green light means go ahead, a green card permits permanent residence in the United States. It is the most important color in Islam. It was the color of the banner of Muhammad, and is found in the flags of nearly all Islamic countries, and represents the lush vegetation of Paradise. It is also often associated with the culture of Gaelic Ireland, and is a color of the flag of Ireland. Because of its association with nature, it is the color of the environmental movement. Political groups advocating environmental protection and social justice describe themselves as part of the Green movement, some naming themselves Green parties. This has led to similar campaigns in advertising, as companies have sold green, or environmentally friendly, products.",What color shows that Mona Lisa is a noble?,"{'text': [], 'answer_start': []}"
3,572732f9dd62a815002e997d,Late_Middle_Ages,"Around 1300–1350 the Medieval Warm Period gave way to the Little Ice Age. The colder climate resulted in agricultural crises, the first of which is known as the Great Famine of 1315-1317. The demographic consequences of this famine, however, were not as severe as the plagues that occurred later in the century, particularly the Black Death. Estimates of the death rate caused by this epidemic range from one

In [16]:
# Load the BERT base uncased tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
def preprocess_data(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        # Check if the answer is available
        if answer["answer_start"] and answer["text"]:
            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])
        else:
            start_char = end_char = 0

        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [17]:
import logging
logging.disable(logging.WARNING)

In [18]:
tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

In [19]:
print(tokenized_dataset['train'][0])

{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}, 'input_ids': [101, 2043, 2106, 20773, 2707, 3352, 2759, 1029, 102, 20773, 21025, 19358, 22815, 1011, 5708, 100

In [20]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = pad_sequence([torch.tensor(item['input_ids']) for item in batch], batch_first=True, padding_value=0)
    attention_mask = pad_sequence([torch.tensor(item['attention_mask']) for item in batch], batch_first=True, padding_value=0)
    start_positions = torch.tensor([item['start_positions'] for item in batch])
    end_positions = torch.tensor([item['end_positions'] for item in batch])

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'start_positions': start_positions, 'end_positions': end_positions}


In [21]:
# Define the model
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

In [36]:
from datasets import load_metric

# Load the metrics
f1_metric = load_metric("f1")
em_metric = load_metric("exact_match")


/scratch/local/26388171/ipykernel_3860057/1295056043.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1_metric = load_metric("f1")
/home/sa.ekbote/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/sa.ekbote/.local/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for exact_match contains custom code which must be executed to correctly load the 

In [43]:
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    start_logits, end_logits = logits
    start_positions = labels[0]
    end_positions = labels[1]

    # Get the predicted start and end positions
    pred_start_positions = np.argmax(start_logits, axis=-1)
    pred_end_positions = np.argmax(end_logits, axis=-1)

    # Calculate the number of exact matches
    exact_matches = np.logical_and(pred_start_positions == start_positions, pred_end_positions == end_positions)
    accuracy = np.mean(exact_matches)

    return {"accuracy": accuracy}

In [44]:
from transformers import TrainerCallback

class ComputeMetricsCallback(TrainerCallback):
    def __init__(self, tokenized_train_dataset):
        self.tokenized_train_dataset = tokenized_train_dataset

    def on_epoch_end(self, args, state, control, **kwargs):
        # Compute metrics on the train dataset
        train_metrics = kwargs['model'].evaluate(self.tokenized_train_dataset)
        print(f"Train metrics: {train_metrics}")

In [45]:
from transformers import Trainer, TrainingArguments, get_linear_schedule_with_warmup
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./bert_squad_v2",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [46]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=3e-5)

num_training_steps = len(tokenized_dataset['train']) // training_args.per_device_train_batch_size * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_training_steps * 0.1,
    num_training_steps=num_training_steps
)


In [48]:

compute_metrics_callback = ComputeMetricsCallback(tokenized_dataset["train"])
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,1.011900,2.039033,0.266908
1000,1.609400,1.403879,0.390718
1500,1.428000,1.240737,0.482439
2000,1.347600,1.202043,0.502148
2500,1.247300,1.144893,0.539965
3000,1.107400,1.167554,0.537185
3500,0.992800,1.179024,0.562705
4000,0.972900,1.108347,0.573907
4500,0.957000,1.063960,0.583930
5000,0.929900,1.058459,0.586036


/home/sa.ekbote/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sa.ekbote/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sa.ekbote/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sa.ekbote/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, bu